In [3]:
###2(a)
import pandas as pd
import numpy as np
from scipy.optimize import newton

# Define the YTM calculation function
def ytm(price, coupon, years_to_maturity, face=100, freq=2):
    coupon_payment = (coupon / 100 * face) / freq
    periods = years_to_maturity * freq
    cash_flows = [coupon_payment] * int(periods)
    cash_flows[-1] += face  # Add face value to last payment

    def pv(y):
        return sum([cf / (1 + y/freq)**t for t, cf in enumerate(cash_flows, 1)]) - price

    return newton(pv, x0=0.05)  # Initial guess YTM = 5%

# Define bond data (from previous selections)
bond_data = [
    ["CA135087Q640", 3.50, 0.58, 100.26],
    ["CA135087R226", 4.50, 1.08, 101.54],
    ["CA135087R556", 4.00, 1.32, 101.24],
    ["CA135087R978", 4.00, 1.58, 101.51],
    ["CA135087S547", 3.00, 2.07, 100.07],
    ["CA135087P733", 3.25, 2.63, 101.00],
    ["CA135087P576", 3.50, 3.14, 101.65],
    ["CA135087Q491", 3.25, 3.65, 101.02],
    ["CA135087Q988", 4.00, 4.14, 103.90],
    ["CA135087R895", 3.50, 4.65, 101.97]
]

# Convert list to DataFrame
bonds_df = pd.DataFrame(bond_data, columns=["ISIN", "Coupon %", "Years to Maturity", "Price"])

# Compute YTM for each bond
bonds_df["YTM (%)"] = bonds_df.apply(lambda row: ytm(row["Price"], row["Coupon %"], row["Years to Maturity"]) * 100, axis=1)

# Display the table in horizontal format
bonds_ytm_table = bonds_df.set_index("ISIN").T

# Show the table to user
import ace_tools as tools
tools.display_dataframe_to_user(name="YTM Calculation Table", dataframe=bonds_ytm_table)

###
# Convert Coupon % from string to numeric format
bonds_df["Coupon %"] = bonds_df["Coupon %"].str.replace("%", "").astype(float)

# Extract price columns (date columns)
price_columns = [col for col in bonds_df.columns if '/' in col]

# Function to calculate YTM
def ytm(price, coupon, years_to_maturity, face_value=100, freq=2):
    """
    Solve for Yield to Maturity using bond pricing formula.
    """
    coupon_payment = (coupon / 100) * face_value / freq
    
    def equation(y):
        return sum([coupon_payment / (1 + y/freq)**(freq*t) for t in range(1, int(years_to_maturity*freq) + 1)]) + \
               face_value / (1 + y/freq)**(freq*years_to_maturity) - price
    
    try:
        return fsolve(equation, x0=0.05)[0] * 100  # Convert to percentage
    except:
        return np.nan  # Return NaN if solving fails

# Dictionary to store yield data
yield_curves = {}

# Calculate YTM for each bond at each date
for date in price_columns:
    yield_data = []
    for _, row in bonds_df.iterrows():
        price = row[date]
        coupon = row["Coupon %"]
        years_to_maturity = row["Years to Maturity"]
        ytm_value = ytm(price, coupon, years_to_maturity)  # Compute YTM
        yield_data.append((years_to_maturity, ytm_value))
    
    yield_curves[date] = sorted(yield_data)  # Store sorted by maturity

# Plot the yield curves
plt.figure(figsize=(10, 6))
for date, curve in yield_curves.items():
    maturities, yields = zip(*curve)
    plt.plot(maturities, yields, marker='o', label=date)
    
plt.xlabel("Years to Maturity")
plt.ylabel("Yield to Maturity (%)")
plt.title("Yield Curve for Selected Bonds")
plt.legend()
plt.grid()
plt.show()


ModuleNotFoundError: No module named 'ace_tools'

In [5]:
###2.2 4(b)
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

# Extract relevant data
dates = bonds_df.columns[6:]  # Extracting the date columns
maturities = bonds_df["Years to Maturity"].values  # Years to maturity
coupon_rates = bonds_df["Coupon %"].str.rstrip('%').astype(float) / 100  # Convert to decimal format
face_value = 100  # Assumed face value of the bond
freq = 2  # Semiannual payments

# Function to solve for spot rates using bootstrapping
def bootstrap_spot_curve(prices, maturities, coupon_rates, face_value=100, freq=2):
    spot_rates = []

    for i in range(len(maturities)):
        P = prices[i]
        C = coupon_rates[i] * face_value / freq
        n = int(maturities[i] * freq)  # Number of periods
        
        if n == 0:
            continue  # Skip bonds with 0 years to maturity

        if i == 0 and C == 0:  # Zero-coupon bond case
            r_n = ((face_value / P) ** (1 / n) - 1) * freq
        else:
            # Solve for r_n using known spot rates
            def bond_price_equation(r_n):
                discount_sum = sum([C / (1 + spot_rates[j] / freq) ** ((j + 1)) for j in range(i)])
                return discount_sum + (face_value + C) / (1 + r_n / freq) ** n - P

            r_n_guess = 0.05  # Initial guess for the root solver
            r_n = fsolve(bond_price_equation, r_n_guess)[0]

        spot_rates.append(r_n)

    return spot_rates

# Compute spot curves for each day
spot_curves = {}

for date in dates:
    prices = bonds_df[date].values
    spot_rates = bootstrap_spot_curve(prices, maturities, coupon_rates, face_value, freq)
    spot_curves[date] = spot_rates

# Plot spot curves
plt.figure(figsize=(10, 6))

for date, rates in spot_curves.items():
    plt.plot(maturities[:len(rates)], rates, label=date, alpha=0.5)

plt.xlabel("Years to Maturity")
plt.ylabel("Spot Rate (%)")
plt.title("5-Year Spot Curve for Each Day")
plt.legend(fontsize=8, loc="upper right", ncol=2)
plt.grid(True)
plt.show()


YTM: 2.93%


In [ ]:
###2.2 4(c)
# Compute 1-year forward rates for maturities from 2 to 5 years
forward_curves = {}

for date, spot_rates in spot_curves.items():
    forward_rates = []

    for n in range(2, 6):  # Compute forward rates for n=2,3,4,5
        if n <= len(spot_rates):  # Ensure we have enough spot rates
            r_n = spot_rates[n - 1]  # Spot rate for maturity n
            r_1 = spot_rates[0]  # 1-year spot rate

            f_1n = freq * (( (1 + r_n / freq) ** (n * freq) / (1 + r_1 / freq) ** freq ) ** (1 / ((n - 1) * freq)) - 1)
            forward_rates.append(f_1n)

    forward_curves[date] = forward_rates

# Plot forward curves
plt.figure(figsize=(10, 6))

for date, rates in forward_curves.items():
    plt.plot(range(2, 6), rates, label=date, alpha=0.5)

plt.xlabel("Years to Maturity (Starting at Year 1)")
plt.ylabel("Forward Rate (%)")
plt.title("1-Year Forward Curve for Each Day")
plt.legend(fontsize=8, loc="upper right", ncol=2)
plt.grid(True)
plt.show()


In [7]:
###Q5
# Step 1: Compute Log-Returns for Yields (9x5 matrix)
log_returns_yield_matrix = np.log(spot_rate_data.shift(-1) / spot_rate_data).dropna()
log_returns_yield_matrix = log_returns_yield_matrix.iloc[:, :5]  # Ensure it's 9x5

# Step 2: Compute Log-Returns for Forward Rates (9x4 matrix)
log_returns_forward_matrix = np.log(forward_rate_data.shift(-1) / forward_rate_data).dropna()
log_returns_forward_matrix = log_returns_forward_matrix.iloc[:, :4]  # Ensure it's 9x4

# Step 3: Compute Covariance Matrices
cov_matrix_yields = log_returns_yield_matrix.cov()  # 5x5 matrix
cov_matrix_forwards = log_returns_forward_matrix.cov()  # 4x4 matrix

# Rename Columns for Clarity
log_returns_yield_matrix.columns = [f'Yield_{i+1}yr' for i in range(log_returns_yield_matrix.shape[1])]
log_returns_forward_matrix.columns = [f'Fwd_{i+1}yr' for i in range(log_returns_forward_matrix.shape[1])]

cov_matrix_yields.columns, cov_matrix_yields.index = log_returns_yield_matrix.columns, log_returns_yield_matrix.columns
cov_matrix_forwards.columns, cov_matrix_forwards.index = log_returns_forward_matrix.columns, log_returns_forward_matrix.columns

# Step 4: Display Results
tools.display_dataframe_to_user(name="Log-Return Matrix for Yields (9x5)", dataframe=log_returns_yield_matrix)
tools.display_dataframe_to_user(name="Log-Return Matrix for Forward Rates (9x4)", dataframe=log_returns_forward_matrix)
tools.display_dataframe_to_user(name="Covariance Matrix of Yields (5x5)", dataframe=cov_matrix_yields)
tools.display_dataframe_to_user(name="Covariance Matrix of Forward Rates (4x4)", dataframe


SyntaxError: incomplete input (2348183745.py, line 24)